# Prompt Engineering

## Imports

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from datasets import load_dataset
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, multilabel_confusion_matrix

c:\Users\rpovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading the Dataset


In [4]:
ds = load_dataset("higopires/RePro-categories-multilabel")

def filter_inadequada(example):
    return example["INADEQUADA"] == 0

ds = ds.filter(filter_inadequada)

test_dataset = ds["test"]

categories = ["ENTREGA", "OUTROS", "PRODUTO", "CONDICOESDERECEBIMENTO", "ANUNCIO"]

## Choosing the Language Model

In [5]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

## Designing the Prompt

In [ ]:

#Zero Shot Prompting
def create_prompt_zero_shot(review_text, categories_list):
    prompt_template = (
        "Sua tarefa é classificar a avaliação do cliente em **até 3 (três) categorias MAIS RELEVANTES** da lista: {}. " 
        "Siga estas regras ESTRITAS para a sua resposta:\n" 
        "1. Retorne APENAS os nomes das categorias da lista, separados por vírgula.\n"
        "3. **NÃO EXCEDA MAIS DO QUE 3 CATEGORIAS.** Se 1 ou 2 categorias forem mais apropriadas, use menos. Priorize as mais importantes.\n" 
        "4. Se NENHUMA categoria da lista se aplicar, retorne uma string vazia (NADA).\n" 
        "5. NÃO adicione introduções, explicações, ou qualquer texto além das categorias (ou string vazia).\n\n"
        "Avaliação: {}\n" 
        "Categorias aplicáveis:" 
    )
    prompt = prompt_template.format(', '.join(categories_list), review_text)
    return prompt

In [7]:
def create_prompt_few_shot(review_text, categories_list):
    examples = [
        {
            "review": "Eu ameiii o produto, pena que veio com o espelho quebrado.",
            "categories": "PRODUTO, CONDICOESDERECEBIMENTO"
        },
        {
            "review": "Quero saber quando chegará mais pois gostaria de comprar",
            "categories": "OUTROS"
        },
        {
            "review": "O cobre leito é de um tecido muito inferior e a cor é totalmente diferente do da foto",
            "categories": "PRODUTO, ANUNCIO"
        },
        {
            "review": "Muito boa Super recomendado. A entrega foi super rápido.", 
            "categories": "ENTREGA" 
        },
        {
            "review": "Recomendo!Os aparelhos da motorola são muito bons!!!",
            "categories": "PRODUTO"
        }
    ]

    prompt_header = (
        "Sua tarefa é classificar a avaliação do cliente em **até 3 (três) categorias MAIS RELEVANTES** da lista: {}. " 
        "Siga estas regras ESTRITAS para a sua resposta:\n" 
        "1. Retorne APENAS os nomes das categorias da lista, separados por vírgula.\n"
        "2. Os nomes devem ser EXATAMENTE como na lista (ex: ENTREGA, PRODUTO). Preste atenção em maiúsculas/minúsculas.\n"
        "3. **NÃO exceda 3 categorias.** Se 1 ou 2 categorias forem mais apropriadas, use menos. Priorize as mais importantes.\n" 
        "4. Se NENHUMA categoria da lista se aplicar, retorne uma string vazia (NADA).\n" 
        "5. NÃO adicione introduções, explicações, ou qualquer texto além das categorias (ou string vazia).\n\n"
        "A seguir, veja exemplos de como formatar a resposta e selecionar as categorias:\n\n" 
    ).format(', '.join(categories_list))


    prompt_parts = [prompt_header] 

    for ex in examples:
        prompt_parts.append(f"Avaliação: \"{ex['review']}\"\nCategorias aplicáveis: {ex['categories']}\n\n")
    
    prompt_parts.append(f"Avaliação: \"{review_text}\"\nCategorias aplicáveis:")
    

    return "".join(prompt_parts)

print(create_prompt_few_shot("A entrega foi rápida e o produto chegou em perfeito estado.", categories))


Sua tarefa é classificar a avaliação do cliente em **até 3 (três) categorias MAIS RELEVANTES** da lista: ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO. Siga estas regras ESTRITAS para a sua resposta:
1. Retorne APENAS os nomes das categorias da lista, separados por vírgula.
2. Os nomes devem ser EXATAMENTE como na lista (ex: ENTREGA, PRODUTO). Preste atenção em maiúsculas/minúsculas.
3. **NÃO exceda 3 categorias.** Se 1 ou 2 categorias forem mais apropriadas, use menos. Priorize as mais importantes.
4. Se NENHUMA categoria da lista se aplicar, retorne uma string vazia (NADA).
5. NÃO adicione introduções, explicações, ou qualquer texto além das categorias (ou string vazia).

A seguir, veja exemplos de como formatar a resposta e selecionar as categorias:

Avaliação: "Eu ameiii o produto, pena que veio com o espelho quebrado."
Categorias aplicáveis: PRODUTO, CONDICOESDERECEBIMENTO

Avaliação: "Quero saber quando chegará mais pois gostaria de comprar"
Categorias aplicáveis: OUT

## Generate and Parse Output

In [8]:
batch_size = 8  # Adjust based on your GPU/CPU memory
all_predictions_prompting_binary = []
all_true_labels_binary = []

print(f"Processing {len(test_dataset)} examples from the test set in batches of {batch_size}...")

use_few_shot = False  # Set to False for zero-shot, True for few-shot

for i in range(0, len(test_dataset), batch_size):

    batch_examples_dict = test_dataset[i:i + batch_size]
    
    num_in_batch = len(batch_examples_dict['review_text'])
    
    batch_prompts = []
    batch_current_true_labels = [] 

    for j in range(num_in_batch):
        review_text = batch_examples_dict["review_text"][j]
        
        if use_few_shot:
            prompt = create_prompt_few_shot(review_text, categories)
        else:
            prompt = create_prompt_zero_shot(review_text, categories)


        batch_prompts.append(prompt)
        
        true_label_row = [int(batch_examples_dict[cat][j]) for cat in categories]
        batch_current_true_labels.append(true_label_row)

    inputs = tokenizer(
        batch_prompts,
        return_tensors="pt",
        padding=True,  
        truncation=True,
        max_length=1000
    ).to(device)
    
    with torch.no_grad(): 
        outputs = model.generate(
            **inputs, 
            max_new_tokens=30,
            num_beams=4,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id,
        )


    generated_texts_batch = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    for k in range(len(generated_texts_batch)):
        generated_text = generated_texts_batch[k]
        
        print(f"Raw LLM Response for review {i+k}: '{generated_text}'")

        true_label_row = batch_current_true_labels[k] 
        
        predicted_category_names = []
        generated_text_strip = generated_text.strip()
        
        if generated_text_strip: 
            categories_upper = {cat.upper(): cat for cat in categories}
            raw_predicted_names = [cat.strip() for cat in generated_text_strip.split(',')]
            for name_raw in raw_predicted_names:
                name_upper = name_raw.upper()
                if name_upper in categories_upper:
                    predicted_category_names.append(categories_upper[name_upper]) 

        binary_prediction = [1 if cat in predicted_category_names else 0 for cat in categories]
        all_predictions_prompting_binary.append(binary_prediction)
        all_true_labels_binary.append(true_label_row) 

    print(f"  Processed batch covering examples {i+1} to {min(i + batch_size, len(test_dataset))}/{len(test_dataset)}.")
    


print("Finished processing all examples.")

y_true_prompting = np.array(all_true_labels_binary)
y_pred_prompting = np.array(all_predictions_prompting_binary)


Processing 966 examples from the test set in batches of 8...
Raw LLM Response for review 0: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO.'
Raw LLM Response for review 1: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO.'
Raw LLM Response for review 2: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO'
Raw LLM Response for review 3: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO.'
Raw LLM Response for review 4: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO.'
Raw LLM Response for review 5: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO.'
Raw LLM Response for review 6: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO'
Raw LLM Response for review 7: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO'
  Processed batch covering examples 1 to 8/966.
Raw LLM Response for review 8: 'ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO'
Raw LLM Response for review 9: 'ENTREGA, OUTROS, PRODUTO, CONDICOES

KeyboardInterrupt: 

## Evaluate and Display Metrics

In [17]:

if len(y_true_prompting) > 0 and len(y_pred_prompting) > 0:
    f1_micro_prompting = f1_score(y_true_prompting, y_pred_prompting, average='micro', zero_division=0)
    precision_micro_prompting = precision_score(y_true_prompting, y_pred_prompting, average='micro', zero_division=0)
    recall_micro_prompting = recall_score(y_true_prompting, y_pred_prompting, average='micro', zero_division=0)

    print(f"\n--- Evaluation Metrics for Prompting Model using Zero Shot ({model_name}) ---")
    print(f"F1 Score (Micro): {f1_micro_prompting:.4f}")
    print(f"Precision (Micro): {precision_micro_prompting:.4f}")
    print(f"Recall (Micro): {recall_micro_prompting:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_true_prompting, y_pred_prompting, target_names=categories, zero_division=0))
else:
    print("Evaluation skipped: No predictions or true labels were generated.")


--- Evaluation Metrics for Prompting Model using Zero Shot (google/flan-t5-base) ---
F1 Score (Micro): 0.5247
Precision (Micro): 0.3614
Recall (Micro): 0.9574

Classification Report:
                        precision    recall  f1-score   support

               ENTREGA       0.31      1.00      0.47       298
                OUTROS       0.23      1.00      0.37       219
               PRODUTO       0.79      1.00      0.88       761
CONDICOESDERECEBIMENTO       0.17      1.00      0.29       164
               ANUNCIO       0.11      0.23      0.14        84

             micro avg       0.36      0.96      0.52      1526
             macro avg       0.32      0.85      0.43      1526
          weighted avg       0.51      0.96      0.62      1526
           samples avg       0.36      0.97      0.51      1526

